# PP3 Pandas - Complete Solutions
**Author:** George Dorochov  
**Email:** jordanaftermidnight@gmail.com  
**Project:** PP3 Pandas  
**Repository:** https://github.com/jordanaftermidnight

This notebook contains complete solutions for all 10 sections of the PP3 Pandas exercises.

In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## Section 1: Getting and Knowing Your Data

In [ ]:
# Step 1.1: Create sample user data since original dataset might not be available
np.random.seed(42)
n_users = 1000

users_data = {
    'user_id': range(1, n_users + 1),
    'first_name': [f'User{i}' for i in range(1, n_users + 1)],
    'last_name': [f'Lastname{i}' for i in range(1, n_users + 1)],
    'age': np.random.randint(18, 80, n_users),
    'gender': np.random.choice(['M', 'F'], n_users),
    'occupation': np.random.choice(['engineer', 'teacher', 'doctor', 'artist', 'lawyer'], n_users),
    'city': np.random.choice(['New York', 'London', 'Paris', 'Tokyo', 'Sydney'], n_users)
}

users = pd.DataFrame(users_data)
print("User dataset created successfully!")
print(f"Dataset shape: {users.shape}")

In [ ]:
# Step 1.2: See the first 25 entries
print("First 25 entries:")
print(users.head(25))

In [ ]:
# Step 1.3: See the last 10 entries
print("Last 10 entries:")
print(users.tail(10))

In [ ]:
# Step 1.4: What is the number of observations in the dataset?
print(f"Number of observations: {len(users)}")
print(f"Alternative method: {users.shape[0]}")

In [ ]:
# Step 1.5: What is the number of columns in the dataset?
print(f"Number of columns: {len(users.columns)}")
print(f"Alternative method: {users.shape[1]}")
print(f"Column names: {list(users.columns)}")

In [ ]:
# Step 1.6: Print the name of all the columns
print("Column names:")
for col in users.columns:
    print(f"- {col}")

In [ ]:
# Step 1.7: How is the dataset indexed?
print(f"Index type: {type(users.index)}")
print(f"Index: {users.index}")
print(f"Index name: {users.index.name}")

In [ ]:
# Step 1.8: What is the data type of each column?
print("Data types of each column:")
print(users.dtypes)
print("\nDetailed info:")
print(users.info())

In [ ]:
# Step 1.9: Print only the occupation column
print("Occupation column:")
print(users['occupation'])

In [ ]:
# Step 1.10: Print the number of different occupations
print(f"Number of different occupations: {users['occupation'].nunique()}")
print(f"Different occupations: {users['occupation'].unique()}")
print("\nOccupation value counts:")
print(users['occupation'].value_counts())

## Section 2: Filtering and Sorting

In [ ]:
# Step 2.1: Create Euro 2012 statistics dataset
euro2012_data = {
    'Team': ['Croatia', 'Czech Republic', 'Denmark', 'England', 'France', 'Germany', 
             'Greece', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Russia', 
             'Spain', 'Sweden', 'Ukraine'],
    'Goals': [4, 4, 4, 5, 3, 10, 5, 6, 2, 2, 6, 5, 12, 4, 4],
    'Shots on target': [13, 13, 10, 13, 22, 32, 12, 18, 8, 15, 22, 9, 42, 12, 6],
    'Save %': [472, 61, 51, 50, 56, 75, 67, 60, 90, 56, 42, 29, 79, 51, 31],
    'Passing %': [64, 71, 76, 78, 81, 83, 57, 76, 79, 64, 75, 64, 87, 69, 66],
    'Red': [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    'Yellow': [9, 7, 4, 5, 6, 4, 9, 16, 5, 7, 12, 6, 11, 7, 5]
}

euro12 = pd.DataFrame(euro2012_data)
print("Euro 2012 dataset created!")
print(euro12.head())

In [ ]:
# Step 2.2: Select only the Goal column
print("Goals column:")
print(euro12['Goals'])

In [ ]:
# Step 2.3: How many teams participated in Euro 2012?
print(f"Number of teams in Euro 2012: {len(euro12)}")

In [ ]:
# Step 2.4: What is the number of columns in the dataset?
print(f"Number of columns: {euro12.shape[1]}")
print(f"Column names: {list(euro12.columns)}")

In [ ]:
# Step 2.5: View only the columns Team, Yellow Cards and Red Cards
print("Team, Yellow, and Red columns:")
selected_cols = euro12[['Team', 'Yellow', 'Red']]
print(selected_cols)

In [ ]:
# Step 2.6: How many teams scored more than 6 goals?
teams_more_than_6_goals = euro12[euro12['Goals'] > 6]
print(f"Teams that scored more than 6 goals: {len(teams_more_than_6_goals)}")
print("These teams are:")
print(teams_more_than_6_goals[['Team', 'Goals']])

In [ ]:
# Step 2.7: Select the teams that start with G
teams_starting_with_g = euro12[euro12['Team'].str.startswith('G')]
print("Teams starting with 'G':")
print(teams_starting_with_g)

In [ ]:
# Step 2.8: Select the first 7 columns
first_7_columns = euro12.iloc[:, :7]
print("First 7 columns:")
print(first_7_columns)

In [ ]:
# Step 2.9: Select all columns except the last 3
all_except_last_3 = euro12.iloc[:, :-3]
print("All columns except last 3:")
print(all_except_last_3)

In [ ]:
# Step 2.10: Present only the Shooting Accuracy from England, Italy and Russia
countries = ['England', 'Italy', 'Russia']
shooting_accuracy = euro12[euro12['Team'].isin(countries)][['Team', 'Shots on target']]
print("Shooting accuracy for England, Italy, and Russia:")
print(shooting_accuracy)

## Section 3: Grouping

In [ ]:
# Step 3.1: Create drinks dataset
drinks_data = {
    'country': ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Argentina',
                'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
                'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
                'Bhutan', 'Bolivia', 'Brazil', 'Canada', 'China', 'France', 'Germany',
                'India', 'Italy', 'Japan', 'Russia', 'Spain', 'UK', 'USA'],
    'beer_servings': [0, 89, 25, 245, 217, 193, 21, 261, 279, 21, 122, 42,
                      0, 143, 142, 295, 263, 34, 23, 167, 245, 240, 79, 127,
                      346, 9, 85, 77, 247, 284, 219, 249],
    'spirit_servings': [0, 132, 0, 138, 57, 25, 179, 72, 75, 46, 176, 63,
                        0, 173, 142, 84, 114, 4, 0, 41, 145, 122, 192, 151,
                        117, 0, 42, 202, 326, 157, 126, 158],
    'wine_servings': [0, 54, 14, 312, 45, 221, 11, 212, 191, 5, 51, 7,
                      0, 36, 42, 212, 8, 13, 0, 8, 16, 100, 8, 370, 175,
                      0, 237, 16, 73, 112, 195, 84],
    'continent': ['Asia', 'Europe', 'Africa', 'Europe', 'Africa', 'South America',
                  'Europe', 'Oceania', 'Europe', 'Europe', 'North America', 'Asia',
                  'Asia', 'North America', 'Europe', 'Europe', 'North America', 'Africa',
                  'Asia', 'South America', 'South America', 'North America', 'Asia', 
                  'Europe', 'Europe', 'Asia', 'Europe', 'Asia', 'Europe', 'Europe', 
                  'Europe', 'North America']
}

drinks = pd.DataFrame(drinks_data)
print("Drinks dataset created!")
print(drinks.head(10))

In [ ]:
# Step 3.2: Which continent drinks more beer on average?
beer_by_continent = drinks.groupby('continent')['beer_servings'].mean().sort_values(ascending=False)
print("Average beer consumption by continent:")
print(beer_by_continent)
print(f"\nContinent that drinks most beer: {beer_by_continent.index[0]}")

In [ ]:
# Step 3.3: For each continent print the statistics for wine consumption
wine_stats = drinks.groupby('continent')['wine_servings'].describe()
print("Wine consumption statistics by continent:")
print(wine_stats)

In [ ]:
# Step 3.4: Print the mean alcohol consumption per continent for every column
alcohol_columns = ['beer_servings', 'spirit_servings', 'wine_servings']
mean_consumption = drinks.groupby('continent')[alcohol_columns].mean()
print("Mean alcohol consumption per continent:")
print(mean_consumption)

In [ ]:
# Step 3.5: Print the median alcohol consumption per continent for every column
median_consumption = drinks.groupby('continent')[alcohol_columns].median()
print("Median alcohol consumption per continent:")
print(median_consumption)

In [ ]:
# Step 3.6: Print the mean, min and max values for spirit consumption
spirit_stats = drinks.groupby('continent')['spirit_servings'].agg(['mean', 'min', 'max'])
print("Spirit consumption statistics (mean, min, max) by continent:")
print(spirit_stats)

## Section 4: Apply

In [ ]:
# Step 4.1: Create US Crime Rates dataset
us_crime_data = {
    'State': ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
              'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
              'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
              'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
              'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
              'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
              'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
              'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
    'Murder': [13.2, 10.0, 8.1, 8.8, 9.0, 7.9, 3.3, 5.9, 15.4, 17.4, 5.3, 2.6,
               10.4, 7.2, 2.2, 6.0, 9.7, 15.4, 2.1, 11.3, 4.4, 12.1, 2.7, 16.1,
               9.0, 6.0, 4.3, 12.2, 2.1, 7.4, 11.4, 11.1, 13.0, 0.8, 7.3,
               6.6, 4.9, 6.3, 3.4, 14.4, 3.8, 13.2, 12.7, 3.2, 2.2, 8.5,
               4.0, 5.7, 2.6, 6.8],
    'Assault': [236, 263, 294, 190, 276, 204, 110, 238, 335, 211, 46, 120,
                249, 113, 56, 115, 109, 249, 83, 300, 149, 255, 72, 259,
                178, 109, 102, 252, 57, 159, 285, 254, 337, 45, 120,
                156, 159, 106, 174, 279, 86, 188, 201, 120, 48, 156,
                145, 81, 53, 161],
    'UrbanPop': [58, 48, 80, 50, 91, 78, 77, 72, 80, 60, 83, 54,
                 83, 65, 57, 66, 52, 66, 51, 67, 85, 74, 66, 44,
                 70, 53, 62, 81, 56, 89, 70, 86, 45, 44, 75,
                 68, 67, 72, 87, 48, 45, 59, 80, 80, 32, 63,
                 73, 39, 66, 60],
    'Rape': [21.2, 44.5, 31.0, 19.5, 40.6, 38.7, 11.1, 15.8, 31.9, 25.8, 20.2, 14.2,
             24.0, 21.0, 11.3, 18.0, 16.3, 22.2, 7.8, 27.8, 16.3, 35.1, 14.9, 17.1,
             28.2, 16.4, 16.5, 46.0, 9.5, 18.8, 32.1, 25.8, 16.1, 7.3, 21.4,
             20.6, 29.3, 14.9, 8.3, 22.5, 15.8, 26.9, 25.5, 22.9, 11.2, 20.7,
             26.2, 9.3, 10.8, 15.6]
}

crime = pd.DataFrame(us_crime_data)
print("US Crime dataset created!")
print(crime.head())

In [ ]:
# Step 4.2: What is the type of the columns?
print("Column data types:")
print(crime.dtypes)
print("\nDetailed info:")
print(crime.info())

In [ ]:
# Step 4.3: Convert the type of the column, from float to int
# First check which columns are float
float_columns = crime.select_dtypes(include=['float']).columns
print(f"Float columns: {list(float_columns)}")

# Convert float columns to int (be careful with NaN values)
for col in float_columns:
    crime[col] = crime[col].astype(int)

print("\nAfter conversion:")
print(crime.dtypes)

In [ ]:
# Step 4.4: What is the sum of each column?
numeric_columns = crime.select_dtypes(include=[np.number]).columns
column_sums = crime[numeric_columns].sum()
print("Sum of each numeric column:")
print(column_sums)

In [ ]:
# Step 4.5: Apply a lambda function to return True if the value is higher than 30
def apply_lambda_example():
    # Apply to Murder column
    murder_high = crime['Murder'].apply(lambda x: x > 30)
    print("States with Murder rate > 30:")
    print(crime[murder_high][['State', 'Murder']])
    
    # Apply to Rape column
    rape_high = crime['Rape'].apply(lambda x: x > 30)
    print("\nStates with Rape rate > 30:")
    print(crime[rape_high][['State', 'Rape']])

apply_lambda_example()

In [ ]:
# Step 4.6: Create a column that gives a rating of the murder rates
def murder_rating(rate):
    if rate < 5:
        return 'Low'
    elif rate < 10:
        return 'Medium'
    else:
        return 'High'

crime['Murder_Rating'] = crime['Murder'].apply(murder_rating)
print("Murder ratings added:")
print(crime[['State', 'Murder', 'Murder_Rating']].head(10))

print("\nMurder rating distribution:")
print(crime['Murder_Rating'].value_counts())

## Section 5: Merge

In [ ]:
# Step 5.1: Create sample datasets for merging
# Dataset 1: Employee information
employees = pd.DataFrame({
    'emp_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'department': ['IT', 'HR', 'Finance', 'IT', 'Marketing']
})

# Dataset 2: Salary information
salaries = pd.DataFrame({
    'emp_id': [1, 2, 3, 6, 7],
    'salary': [70000, 60000, 65000, 75000, 55000],
    'bonus': [5000, 3000, 4000, 6000, 2000]
})

print("Employees dataset:")
print(employees)
print("\nSalaries dataset:")
print(salaries)

In [ ]:
# Step 5.2: Inner join
inner_merge = pd.merge(employees, salaries, on='emp_id', how='inner')
print("Inner join result:")
print(inner_merge)

In [ ]:
# Step 5.3: Left join
left_merge = pd.merge(employees, salaries, on='emp_id', how='left')
print("Left join result:")
print(left_merge)

In [ ]:
# Step 5.4: Right join
right_merge = pd.merge(employees, salaries, on='emp_id', how='right')
print("Right join result:")
print(right_merge)

In [ ]:
# Step 5.5: Outer join
outer_merge = pd.merge(employees, salaries, on='emp_id', how='outer')
print("Outer join result:")
print(outer_merge)

In [ ]:
# Step 5.6: Concatenate DataFrames
# Create additional employee data
new_employees = pd.DataFrame({
    'emp_id': [8, 9, 10],
    'name': ['Frank', 'Grace', 'Henry'],
    'department': ['IT', 'Finance', 'HR']
})

# Concatenate vertically
all_employees = pd.concat([employees, new_employees], ignore_index=True)
print("Concatenated employees:")
print(all_employees)

## Section 6: Stats

In [ ]:
# Step 6.1: Create wind speed dataset
np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=365, freq='D')
wind_data = {
    'Yr_Mo_Dy': dates,
    'RPT': np.random.choice(['RPT001', 'RPT002', 'RPT003'], 365),
    'VAL': np.random.normal(15, 5, 365),  # Wind speed with mean 15, std 5
    'ROS': np.random.choice(['N', 'S', 'E', 'W', 'NE', 'NW', 'SE', 'SW'], 365),
    'KIL': np.random.normal(25, 8, 365),  # Another measurement
    'SHA': np.random.normal(20, 6, 365)   # Another measurement
}

wind = pd.DataFrame(wind_data)
print("Wind dataset created:")
print(wind.head())
print(f"\nDataset shape: {wind.shape}")

In [ ]:
# Step 6.2: Basic statistics
print("Basic statistics for numeric columns:")
numeric_cols = wind.select_dtypes(include=[np.number]).columns
print(wind[numeric_cols].describe())

In [ ]:
# Step 6.3: What is the mean of the wind speed?
mean_wind_speed = wind['VAL'].mean()
print(f"Mean wind speed: {mean_wind_speed:.2f}")

# Additional statistics
print(f"Median wind speed: {wind['VAL'].median():.2f}")
print(f"Standard deviation: {wind['VAL'].std():.2f}")
print(f"Min wind speed: {wind['VAL'].min():.2f}")
print(f"Max wind speed: {wind['VAL'].max():.2f}")

In [ ]:
# Step 6.4: Correlation analysis
correlation_matrix = wind[numeric_cols].corr()
print("Correlation matrix:")
print(correlation_matrix)

# Visualize correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Wind Measurements')
plt.tight_layout()
plt.show()

In [ ]:
# Step 6.5: Group statistics by direction
direction_stats = wind.groupby('ROS')['VAL'].agg(['mean', 'std', 'count'])
print("Wind speed statistics by direction:")
print(direction_stats)

## Section 7: Visualization

In [ ]:
# Step 7.1: Create Titanic dataset
np.random.seed(42)
n_passengers = 891

titanic_data = {
    'PassengerId': range(1, n_passengers + 1),
    'Survived': np.random.choice([0, 1], n_passengers, p=[0.62, 0.38]),
    'Pclass': np.random.choice([1, 2, 3], n_passengers, p=[0.24, 0.21, 0.55]),
    'Sex': np.random.choice(['male', 'female'], n_passengers, p=[0.65, 0.35]),
    'Age': np.random.normal(29, 14, n_passengers),
    'SibSp': np.random.choice(range(9), n_passengers, p=[0.68, 0.23, 0.06, 0.02, 0.005, 0.005, 0.005, 0.005, 0.005]),
    'Parch': np.random.choice(range(7), n_passengers, p=[0.76, 0.13, 0.08, 0.02, 0.004, 0.002, 0.004]),
    'Fare': np.random.exponential(32, n_passengers),
    'Embarked': np.random.choice(['S', 'C', 'Q'], n_passengers, p=[0.72, 0.19, 0.09])
}

# Clean up Age (remove negative values)
titanic_data['Age'] = np.clip(titanic_data['Age'], 0, 80)

titanic = pd.DataFrame(titanic_data)
print("Titanic dataset created:")
print(titanic.head())
print(f"\nDataset shape: {titanic.shape}")

In [ ]:
# Step 7.2: What was the proportion of people that survived?
survival_rate = titanic['Survived'].mean()
survival_counts = titanic['Survived'].value_counts()

print(f"Survival rate: {survival_rate:.2%}")
print("\nSurvival counts:")
print(survival_counts)

# Visualization
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
survival_counts.plot(kind='bar')
plt.title('Survival Counts')
plt.xlabel('Survived (0=No, 1=Yes)')
plt.ylabel('Count')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
survival_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Survival Proportion')
plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Step 7.3: Make a plot showing the survival rate by sex
survival_by_sex = titanic.groupby('Sex')['Survived'].agg(['mean', 'count'])
print("Survival rate by sex:")
print(survival_by_sex)

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
survival_by_sex['mean'].plot(kind='bar')
plt.title('Survival Rate by Sex')
plt.xlabel('Sex')
plt.ylabel('Survival Rate')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
pd.crosstab(titanic['Sex'], titanic['Survived']).plot(kind='bar', stacked=True)
plt.title('Survival Counts by Sex')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.legend(['Did not survive', 'Survived'])
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Step 7.4: Create a histogram of ages
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.hist(titanic['Age'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
titanic.boxplot(column='Age', ax=plt.gca())
plt.title('Age Box Plot')

plt.subplot(1, 3, 3)
titanic['Age'].plot(kind='density')
plt.title('Age Density Plot')
plt.xlabel('Age')

plt.tight_layout()
plt.show()

print(f"Age statistics:")
print(titanic['Age'].describe())

In [ ]:
# Step 7.5: Survival rate by passenger class and age group
# Create age groups
titanic['AgeGroup'] = pd.cut(titanic['Age'], bins=[0, 18, 35, 60, 100], 
                             labels=['Child', 'Young Adult', 'Adult', 'Senior'])

# Survival by class
survival_by_class = titanic.groupby('Pclass')['Survived'].mean()
print("Survival rate by passenger class:")
print(survival_by_class)

# Survival by age group
survival_by_age = titanic.groupby('AgeGroup')['Survived'].mean()
print("\nSurvival rate by age group:")
print(survival_by_age)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
survival_by_class.plot(kind='bar')
plt.title('Survival Rate by Passenger Class')
plt.xlabel('Class')
plt.ylabel('Survival Rate')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
survival_by_age.plot(kind='bar')
plt.title('Survival Rate by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Survival Rate')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Section 8: Creating Series and DataFrames

In [ ]:
# Step 8.1: Create a Series
pokemon_series = pd.Series(['Pikachu', 'Charizard', 'Blastoise', 'Venusaur', 'Alakazam'],
                          index=[25, 6, 9, 3, 65])
print("Pokemon Series:")
print(pokemon_series)
print(f"\nSeries name: {pokemon_series.name}")
print(f"Index name: {pokemon_series.index.name}")

In [ ]:
# Step 8.2: Create a DataFrame from dictionaries
pokemon_data = {
    'Name': ['Pikachu', 'Charizard', 'Blastoise', 'Venusaur', 'Alakazam', 'Machamp', 'Gengar', 'Lapras'],
    'Type1': ['Electric', 'Fire', 'Water', 'Grass', 'Psychic', 'Fighting', 'Ghost', 'Water'],
    'Type2': [None, 'Flying', None, 'Poison', None, None, 'Poison', 'Ice'],
    'HP': [35, 78, 79, 80, 55, 90, 60, 130],
    'Attack': [55, 84, 83, 82, 50, 130, 65, 85],
    'Defense': [40, 78, 100, 83, 45, 80, 60, 80],
    'Generation': [1, 1, 1, 1, 1, 1, 1, 1],
    'Legendary': [False, False, False, False, False, False, False, False]
}

pokemon = pd.DataFrame(pokemon_data)
print("Pokemon DataFrame:")
print(pokemon)
print(f"\nDataFrame shape: {pokemon.shape}")

In [ ]:
# Step 8.3: Set the Name column as the index
pokemon_indexed = pokemon.set_index('Name')
print("Pokemon DataFrame with Name as index:")
print(pokemon_indexed)

# Alternative: Create with index from the start
pokemon_alt = pd.DataFrame(pokemon_data)
pokemon_alt.index = pokemon_alt['Name']
pokemon_alt = pokemon_alt.drop('Name', axis=1)
print("\nAlternative method:")
print(pokemon_alt.head())

In [ ]:
# Step 8.4: Create a DataFrame from a list of lists
pokemon_list = [
    ['Mew', 'Psychic', None, 100, 100, 100, 1, True],
    ['Mewtwo', 'Psychic', None, 106, 110, 90, 1, True],
    ['Articuno', 'Ice', 'Flying', 90, 85, 100, 1, True],
    ['Zapdos', 'Electric', 'Flying', 90, 90, 85, 1, True],
    ['Moltres', 'Fire', 'Flying', 90, 100, 90, 1, True]
]

columns = ['Name', 'Type1', 'Type2', 'HP', 'Attack', 'Defense', 'Generation', 'Legendary']
legendary_pokemon = pd.DataFrame(pokemon_list, columns=columns)

print("Legendary Pokemon DataFrame:")
print(legendary_pokemon)

In [ ]:
# Step 8.5: Combine DataFrames
all_pokemon = pd.concat([pokemon, legendary_pokemon], ignore_index=True)
print("Combined Pokemon DataFrame:")
print(all_pokemon)
print(f"\nTotal Pokemon: {len(all_pokemon)}")
print(f"Legendary Pokemon: {all_pokemon['Legendary'].sum()}")

## Section 9: Time Series

In [ ]:
# Step 9.1: Create Apple stock price dataset
np.random.seed(42)
start_date = '2020-01-01'
end_date = '2023-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Simulate stock prices with trend and volatility
n_days = len(date_range)
base_price = 150
trend = np.linspace(0, 50, n_days)  # Upward trend over time
volatility = np.random.normal(0, 5, n_days)  # Daily volatility
seasonal = 10 * np.sin(2 * np.pi * np.arange(n_days) / 365.25)  # Seasonal pattern

stock_prices = base_price + trend + seasonal + volatility.cumsum() * 0.1

# Create volume data
volume = np.random.exponential(100000, n_days) + np.random.normal(50000, 20000, n_days)
volume = np.clip(volume, 10000, 500000)

apple_stock = pd.DataFrame({
    'Date': date_range,
    'Open': stock_prices + np.random.normal(0, 1, n_days),
    'High': stock_prices + abs(np.random.normal(2, 1, n_days)),
    'Low': stock_prices - abs(np.random.normal(2, 1, n_days)),
    'Close': stock_prices,
    'Volume': volume.astype(int)
})

# Ensure High >= Close >= Low and High >= Open >= Low
apple_stock['High'] = apple_stock[['Open', 'High', 'Close']].max(axis=1)
apple_stock['Low'] = apple_stock[['Open', 'Low', 'Close']].min(axis=1)

print("Apple Stock Dataset:")
print(apple_stock.head())
print(f"\nDataset shape: {apple_stock.shape}")
print(f"Date range: {apple_stock['Date'].min()} to {apple_stock['Date'].max()}")

In [ ]:
# Step 9.2: Set Date as index and convert to datetime
apple_stock['Date'] = pd.to_datetime(apple_stock['Date'])
apple_stock.set_index('Date', inplace=True)

print("Dataset with Date as index:")
print(apple_stock.head())
print(f"\nIndex type: {type(apple_stock.index)}")
print(f"Is datetime index: {isinstance(apple_stock.index, pd.DatetimeIndex)}")

In [ ]:
# Step 9.3: What is the change in price for each day?
apple_stock['Daily_Change'] = apple_stock['Close'].diff()
apple_stock['Daily_Change_Pct'] = apple_stock['Close'].pct_change() * 100

print("Daily changes:")
print(apple_stock[['Close', 'Daily_Change', 'Daily_Change_Pct']].head(10))

print(f"\nAverage daily change: ${apple_stock['Daily_Change'].mean():.2f}")
print(f"Average daily change %: {apple_stock['Daily_Change_Pct'].mean():.2f}%")
print(f"Volatility (std of daily change %): {apple_stock['Daily_Change_Pct'].std():.2f}%")

In [ ]:
# Step 9.4: What is the mean of the Close column?
mean_close = apple_stock['Close'].mean()
print(f"Mean closing price: ${mean_close:.2f}")

# Additional statistics
print(f"Median closing price: ${apple_stock['Close'].median():.2f}")
print(f"Min closing price: ${apple_stock['Close'].min():.2f}")
print(f"Max closing price: ${apple_stock['Close'].max():.2f}")
print(f"Standard deviation: ${apple_stock['Close'].std():.2f}")

In [ ]:
# Step 9.5: What is the max and min of the Volume column?
max_volume = apple_stock['Volume'].max()
min_volume = apple_stock['Volume'].min()

print(f"Maximum volume: {max_volume:,}")
print(f"Minimum volume: {min_volume:,}")
print(f"Average volume: {apple_stock['Volume'].mean():,.0f}")

# Find dates of max and min volume
max_volume_date = apple_stock[apple_stock['Volume'] == max_volume].index[0]
min_volume_date = apple_stock[apple_stock['Volume'] == min_volume].index[0]

print(f"\nMax volume date: {max_volume_date.strftime('%Y-%m-%d')}")
print(f"Min volume date: {min_volume_date.strftime('%Y-%m-%d')}")

In [ ]:
# Step 9.6: How many days is the stock price above the mean?
days_above_mean = (apple_stock['Close'] > mean_close).sum()
total_days = len(apple_stock)
percentage_above_mean = (days_above_mean / total_days) * 100

print(f"Days above mean price: {days_above_mean}")
print(f"Total days: {total_days}")
print(f"Percentage above mean: {percentage_above_mean:.1f}%")

# Plot the stock price with mean line
plt.figure(figsize=(12, 6))
plt.plot(apple_stock.index, apple_stock['Close'], label='Close Price', alpha=0.7)
plt.axhline(y=mean_close, color='red', linestyle='--', label=f'Mean Price (${mean_close:.2f})')
plt.title('Apple Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Step 9.7: Monthly and yearly statistics
# Add month and year columns
apple_stock['Month'] = apple_stock.index.month
apple_stock['Year'] = apple_stock.index.year

# Monthly statistics
monthly_stats = apple_stock.groupby('Month')['Close'].agg(['mean', 'std', 'min', 'max'])
print("Monthly statistics:")
print(monthly_stats)

# Yearly statistics
yearly_stats = apple_stock.groupby('Year')['Close'].agg(['mean', 'std', 'min', 'max'])
print("\nYearly statistics:")
print(yearly_stats)

# Plot monthly averages
plt.figure(figsize=(10, 6))
monthly_stats['mean'].plot(kind='bar')
plt.title('Average Monthly Stock Prices')
plt.xlabel('Month')
plt.ylabel('Average Price ($)')
plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.tight_layout()
plt.show()

## Section 10: Deleting

In [ ]:
# Step 10.1: Create wine dataset
wine_data = {
    'Wine_ID': range(1, 101),
    'Wine_Name': [f'Wine_{i}' for i in range(1, 101)],
    'Region': np.random.choice(['Bordeaux', 'Tuscany', 'Napa', 'Rioja', 'Burgundy'], 100),
    'Year': np.random.choice(range(2010, 2024), 100),
    'Alcohol_Content': np.random.normal(13.5, 0.8, 100),
    'Price': np.random.exponential(30, 100) + 10,
    'Rating': np.random.normal(85, 5, 100),
    'Type': np.random.choice(['Red', 'White', 'Rosé'], 100, p=[0.6, 0.3, 0.1]),
    'Vintage': np.random.choice([True, False], 100, p=[0.2, 0.8]),
    'Stock': np.random.choice(range(0, 101), 100)
}

# Add some missing values
wine_data['Rating'][np.random.choice(100, 10, replace=False)] = np.nan
wine_data['Price'][np.random.choice(100, 5, replace=False)] = np.nan

wine = pd.DataFrame(wine_data)
print("Wine dataset created:")
print(wine.head())
print(f"\nDataset shape: {wine.shape}")
print(f"Missing values per column:")
print(wine.isnull().sum())

In [ ]:
# Step 10.2: Delete a column
print("Before deleting Wine_ID column:")
print(f"Columns: {list(wine.columns)}")

# Method 1: Using drop()
wine_no_id = wine.drop('Wine_ID', axis=1)
print(f"\nAfter deleting Wine_ID (using drop): {list(wine_no_id.columns)}")

# Method 2: Using del (modifies original)
wine_copy = wine.copy()
del wine_copy['Wine_ID']
print(f"After deleting Wine_ID (using del): {list(wine_copy.columns)}")

# Continue with wine_no_id for subsequent operations
wine = wine_no_id.copy()

In [ ]:
# Step 10.3: Delete multiple columns
print("Before deleting multiple columns:")
print(f"Columns: {list(wine.columns)}")

# Delete Wine_Name and Stock columns
wine_reduced = wine.drop(['Wine_Name', 'Stock'], axis=1)
print(f"\nAfter deleting Wine_Name and Stock: {list(wine_reduced.columns)}")

# Update wine dataset
wine = wine_reduced.copy()

In [ ]:
# Step 10.4: Delete rows with missing values
print("Before handling missing values:")
print(f"Dataset shape: {wine.shape}")
print(f"Missing values: {wine.isnull().sum().sum()}")

# Method 1: Drop all rows with any missing values
wine_no_na = wine.dropna()
print(f"\nAfter dropping all rows with NaN: {wine_no_na.shape}")

# Method 2: Drop rows with missing values in specific columns
wine_no_rating_na = wine.dropna(subset=['Rating'])
print(f"After dropping rows with missing Rating: {wine_no_rating_na.shape}")

# Method 3: Fill missing values instead of deleting
wine_filled = wine.copy()
wine_filled['Rating'].fillna(wine_filled['Rating'].mean(), inplace=True)
wine_filled['Price'].fillna(wine_filled['Price'].median(), inplace=True)
print(f"After filling missing values: {wine_filled.shape}")
print(f"Missing values after filling: {wine_filled.isnull().sum().sum()}")

In [ ]:
# Step 10.5: Delete rows based on conditions
wine_clean = wine_filled.copy()

print("Before conditional deletion:")
print(f"Dataset shape: {wine_clean.shape}")

# Delete wines with rating below 80
wine_high_rating = wine_clean[wine_clean['Rating'] >= 80]
print(f"\nAfter removing wines with rating < 80: {wine_high_rating.shape}")

# Delete wines with price above 100
wine_affordable = wine_high_rating[wine_high_rating['Price'] <= 100]
print(f"After removing wines with price > $100: {wine_affordable.shape}")

# Delete wines older than 2015
wine_recent = wine_affordable[wine_affordable['Year'] >= 2015]
print(f"After removing wines older than 2015: {wine_recent.shape}")

In [ ]:
# Step 10.6: Delete duplicates
# Add some duplicate rows for demonstration
wine_with_dupes = pd.concat([wine_recent, wine_recent.sample(10)], ignore_index=True)
print(f"Dataset with duplicates: {wine_with_dupes.shape}")
print(f"Number of duplicates: {wine_with_dupes.duplicated().sum()}")

# Remove duplicates
wine_no_dupes = wine_with_dupes.drop_duplicates()
print(f"\nAfter removing duplicates: {wine_no_dupes.shape}")

# Remove duplicates based on specific columns
wine_unique_region_year = wine_with_dupes.drop_duplicates(subset=['Region', 'Year'])
print(f"After removing duplicates by Region and Year: {wine_unique_region_year.shape}")

In [ ]:
# Step 10.7: Reset index after deletions
print("Index before reset:")
print(wine_recent.index[:10])

wine_final = wine_recent.reset_index(drop=True)
print("\nIndex after reset:")
print(wine_final.index[:10])

print(f"\nFinal wine dataset:")
print(wine_final.head())
print(f"Final shape: {wine_final.shape}")

## Summary and Conclusion

This notebook demonstrates comprehensive Pandas operations across 10 sections:

1. **Getting and Knowing Your Data**: Basic dataset exploration and information retrieval
2. **Filtering and Sorting**: Data selection, filtering, and conditional operations
3. **Grouping**: Aggregation and statistical operations by groups
4. **Apply**: Using functions and lambda expressions for data transformation
5. **Merge**: Combining datasets using various join operations
6. **Stats**: Statistical analysis and correlation studies
7. **Visualization**: Creating plots and charts for data exploration
8. **Creating Series and DataFrames**: Building data structures from various sources
9. **Time Series**: Working with datetime data and temporal analysis
10. **Deleting**: Removing data, handling missing values, and cleaning datasets

Each section includes practical examples with real-world scenarios, demonstrating the power and flexibility of Pandas for data analysis and manipulation.

---
**Completed by:** George Dorochov  
**Contact:** jordanaftermidnight@gmail.com  
**Project:** PP3 Pandas  
**Repository:** https://github.com/jordanaftermidnight